In [68]:
import numpy as np
import pandas as pd
import math
from scipy import stats
import matplotlib.pyplot as plt

In [19]:
seed = 42
np.random.seed(seed)

In [99]:
def get_real_distribution():
    """
    generate a dataframe with the mean of the number of births in america from 1994 to 2003
    :return: dataframe with 1 column for the birth rate
    """

    df = pd.read_csv("US_births_1994-2003_CDC_NCHS.csv")
    df = (
        df.groupby(['month', 'date_of_month'])
            .births
            .mean()
    )
    df = df.drop(59)  # remove the 29th of February
    df = pd.DataFrame(df.values, columns=["births"])
    return df

In [4]:
def compute_average(k, n, seed, distribution, distribution_type="uniform"):
    """
    compute the average trials needed to have a collision
    :param k: number of runs
    :param n: cardinality of the set
    :param seed: random seed
    :param distribution: dataframe with the birth distribution
    :param distribution_type: type of the distribution, can be "real" or "uniform"
    :return: list of number of element extracted to have a collision for each run
    """

    if distribution_type != "uniform" and distribution_type != "real":
        print("non valid distribution type")
        return -1

    np.random.seed(seed)  # set random seed
    collision_list = []
    for i in range(k):
        l = []
        m = 0
        while True:
            if distribution_type == "uniform":
                x = np.random.randint(0, n)  # extract a number between 0 and n from a uniform distribution
            else:
                x = distribution.sample(1, weights="births", replace=True).index.values  # extract a number between 0 and n from the real distribution

            if x in l:
                collision_list.append(m)  # if there is a collision save the number of times it took to have one
                break

            l.append(x)  # if there is no collision add the new value to the list and continue
            m += 1  # increase counter

    return collision_list

In [42]:
def compute_probability(m, n, k, seed, distribution, distribution_type="uniform"):
    """
    compute the probability of collision of m elements in a set of cardinality n
    :param m: number of elements
    :param n: cardinality of the set
    :param k: number of runs
    :param seed: random seed
    :param distribution: dataframe with the birth distribution
    :param distribution_type: type of the distribution, can be "real" or "uniform"
    :return: probability of having a collision
    """

    if distribution_type != "uniform" and distribution_type != "real":
        print("non valid distribution type")
        return -1

    np.random.seed(seed)  # set random seed
    collisions = 0
    for i in range(k):
        if distribution_type == "uniform":
            x = np.random.randint(0, n, m)  # extract m number between 0 and n from a uniform distribution
        else:
            x = distribution.sample(m, weights="births", replace=True).index.values  # extract m number between 0 and n from the real distribution

        if len(np.unique(x)) < len(x):  # if there is a collision increment counter
            collisions += 1

    return collisions / k

In [6]:
def expected_p(m, n):
    """
    compute the theoretical value of p
    :param m: number of elements
    :param n: cardinality of the set
    :return: p
    """
    return 1 - math.exp(-math.pow(m, 2) / (2 * n))

In [7]:
def expected_m(n, p):
    """
    compute the theoretical value of m
    :param n: cardinality of the set
    :param p: probability
    :return: m
    """
    return math.sqrt(2 * n * math.log(1 / (1 - p)))

In [8]:
def expected_Em(n):
    """
    compute the theoretical value of the expected value of m
    :param n: cardinality of the set
    :return: E[m]
    """
    return math.sqrt((math.pi / 2) * n)

In [ ]:
def simulation_average(nRuns, n, k):
    """
    simulator of the average trials needed to have a collision, both for the uniform and for the real distribution
    :param nRuns: number of runs (each with a different seed)
    :param n: cardinality of the set
    :param k: number of trials
    :return: dataframe with the result
    """

    df = get_real_distribution()
    d = dict()
    for seed in range(nRuns):
        l_uniform = compute_average(k, n, seed, None, "uniform")
        l_real = compute_average(k, n, seed, df, "real")
        mean_uniform = np.mean(l_uniform)
        mean_real = np.mean(l_real)
        d[seed] = [mean_uniform, mean_real]

    return pd.DataFrame.from_dict(d, orient="index", columns=["uniform", "real"])

In [ ]:
def simulation_probability(nRuns, m, n, k):
    """
    simulator of the probability of having a collision, both for the uniform and for the real distribution
    :param nRuns: number of runs (each with a different seed)
    :param m: number of elements
    :param n: cardinality of the set
    :param k: number of trials
    :return: dataframe with the result
    """

    df = get_real_distribution()
    d = dict()
    for seed in range(nRuns):
        p_uniform = compute_probability(m, n, k, seed, None, "uniform")
        p_real = compute_probability(m, n, k, seed, df, "real")
        d[seed] = [p_uniform, p_real]

    return pd.DataFrame.from_dict(d, orient="index", columns=["uniform", "real"])

In [ ]:
def confidence_interval(df, q):
    """

    :param df: dataframe with measurements 1 column for uniform distribution and 1 column for the real one
    :param q: percentile of the confidence interval
    :return: 2 dataframes one with the lower bound and one with the upper bound for each distribution
    """
    mean = df.mean()
    variance = df.var()

    t_crit = stats.t.ppf(q=q, df=df.shape[0])
    down_i = mean - t_crit * np.sqrt(variance)/np.sqrt(df.shape[0])
    up_i = mean + t_crit * np.sqrt(variance)/np.sqrt(df.shape[0])

    return down_i, up_i

In [9]:
df = get_real_distribution()

In [10]:
l_uniform = compute_average(1000, 365, seed, None, "uniform")
l_real = compute_average(1000, 365, seed, df, "real")

In [11]:
mean_uniform = np.mean(l_uniform)
mean_real = np.mean(l_real)
expected_mean = expected_Em(365)

In [12]:
print(mean_uniform, mean_real, expected_mean)

23.336 23.554 23.944532972687885


In [51]:
p_uniform = compute_probability(23, 365, 1000, seed, None, "uniform")
p_real = compute_probability(23, 365, 1000, seed, df, "real")
_expected_p = expected_p(23, 365)

In [52]:
print(p_uniform, p_real, _expected_p)

0.517 0.523 0.5155095380615168


In [62]:
df_average = simulation_average(10, 365, 100)

In [63]:
df_probability = simulation_probability(10, 23, 365, 1000)

In [81]:
confidence_interval(df_average, .95)

(uniform    23.180231
 real       22.667758
 dtype: float64,
 uniform    24.213769
 real       24.194242
 dtype: float64)

In [82]:
confidence_interval(df_probability, .95)

(uniform    0.497151
 real       0.504742
 dtype: float64,
 uniform    0.520449
 real       0.517858
 dtype: float64)